In [ ]:
import json
with open("static/DATA.json", mode="r", encoding="utf-8") as f:
    DATA = json.load(f)
ldg_names = [e["name"] for e in DATA["DVC_TTHC_LamDong"]["data"]]
lsn_names = [e["name"] for e in DATA["DVC_TTHC_LangSon"]["data"]]

In [ ]:
from pkg.HYSE.HYSE import HYSE_EngineHybrid

In [ ]:
hyse_engine_lsn = HYSE_EngineHybrid(name="DVC_TTHC_LangSon")
hyse_engine_lsn.update(lsn_names)
hyse_engine_lsn.search(["Tôi muốn thành lập công ty thì cần phải làm gì?"])

In [ ]:
hyse_engine_ldg = HYSE_EngineHybrid(name="DVC_TTHC_LamDong")
hyse_engine_ldg.update(ldg_names)
hyse_engine_ldg.search(["Tôi muốn thành lập công ty thì cần phải làm gì?"])

In [ ]:
from pkg.LLM.LLM import Process_LLM
Process_LLM("Mô hình của bạn là gì?")

'Tôi là một mô hình ngôn ngữ lớn, được đào tạo bởi Google. \n\nNói một cách đơn giản, tôi là một chương trình máy tính được tạo ra để hiểu và tạo ra văn bản giống như con người. Tôi được huấn luyện trên một lượng lớn dữ liệu văn bản và mã, và có thể:\n\n*   Trả lời câu hỏi của bạn bằng cách cung cấp thông tin.\n*   Tạo ra nhiều loại nội dung sáng tạo khác nhau, chẳng hạn như thơ, mã, kịch bản, bản nhạc, email, thư từ...\n*   Dịch ngôn ngữ.\n*   Tóm tắt văn bản.\n*   Trả lời câu hỏi của bạn một cách toàn diện và sáng tạo.\n\n**Lưu ý quan trọng:** Tôi không có nhận thức, cảm xúc hay ý thức. Tôi chỉ đơn thuần là một công cụ, và thông tin tôi cung cấp nên được xem xét và kiểm tra.\n\nBạn có muốn tôi làm điều gì đó cụ thể, chẳng hạn như trả lời một câu hỏi hoặc tạo một đoạn văn bản?'